# Dependancies

In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from keras import *

import numpy as np
import pandas as pd

Using TensorFlow backend.


# Data

In [0]:
path_to_data = "/content/Full Database for TCRs.csv"

In [0]:
data = pd.read_csv(path_to_data)
data = data.dropna()
data = data.reset_index(drop=True)

data_epitope = data['Epitope'][:50001]
data_v = data['V'][:50001]
data_j = data['J'][:50001]
data_cdr3 = data['CDR3'][:50001]

In [0]:
def encoder_protien(data_frame):
  encoder_dict = {'a':2,'b':3,'c':4,'d':5,'e':6,'f':7,'g':8,'h':9,'i':10,
             'j':11,'k':12,'l':13,'m':14,'n':15,'o':16,'p':17,'q':18,'r':19,'s':20,
             't':21,'u':22,'v':23,'w':24,'x':25,'y':26,'z':27,'=':28}

  output = []
  max_length = 0

  for sequence in data_frame:
    a = len(sequence)
    if (a > max_length):
      max_length = a+2

  for sequence in data_frame:
    sequence_temp = []

    for protien in sequence:
      sequence_temp.append(encoder_dict[protien.lower()])

    for i in range(max_length-len(sequence_temp)):
      sequence_temp.append(0)

    output.append(sequence_temp)

  output = np.asarray(output)
  return output

In [0]:
from keras.utils import to_categorical
unique_j = list(data_j.unique())

Y_Train = []

for i in data_j:
  temp = []
  Y_Train.append(unique_j.index(i))

Y_Train = to_categorical(Y_Train)

Y_Train = np.asarray(Y_Train)
X_Train = encoder_protien(data_epitope)

In [0]:
Y_Train.shape


(50001, 68)

In [0]:
Input_Layer = tf.keras.layers.Input(shape=(22))

Embedding_Layer = tf.keras.layers.Embedding(22,4000)(Input_Layer)
# LSTM1 = tf.keras.layers.LSTM(400)(Embedding_Layer)

Embedding_Layer = tf.keras.layers.Flatten()(Embedding_Layer)

# Dense0 = tf.keras.layers.Dense(5000,activation='relu')(Embedding_Layer)
Dense1 = tf.keras.layers.Dense(5000,activation='relu')(Embedding_Layer)
Dense2 = tf.keras.layers.Dense(5000,activation='relu')(Dense1)
Dense3 = tf.keras.layers.Dense(2500)(Dense2)
Dense4 = tf.keras.layers.Dense(500)(Dense3) 

Dense5 = tf.keras.layers.Dense(68, activation='softmax')(Dense4)
model = tf.keras.Model(inputs=Input_Layer,outputs=Dense5)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(optimizer='SGD',loss='categorical_crossentropy',metrics=['accuracy'])
saved = tf.keras.callbacks.ModelCheckpoint("/content/TCR_Normal_Model.hdf5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')

In [0]:
model.fit(X_Train,Y_Train,epochs=200,validation_split=0.2,shuffle=True,callbacks=[saved])

Train on 40000 samples, validate on 10001 samples
Epoch 1/200
40000/40000 [==============================] - 59s 1ms/sample - loss: 2.3437 - acc: 0.1937 - val_loss: 6.5089 - val_acc: 0.1234
Epoch 2/200
40000/40000 [==============================] - 41s 1ms/sample - loss: 2.2650 - acc: 0.2125 - val_loss: 6.7798 - val_acc: 0.1188
Epoch 3/200
40000/40000 [==============================] - 41s 1ms/sample - loss: 2.2493 - acc: 0.2175 - val_loss: 6.7696 - val_acc: 0.1227
Epoch 4/200
40000/40000 [==============================] - 40s 1ms/sample - loss: 2.2391 - acc: 0.2193 - val_loss: 7.2222 - val_acc: 0.1230
Epoch 5/200
40000/40000 [==============================] - 40s 1ms/sample - loss: 2.2334 - acc: 0.2212 - val_loss: 6.9800 - val_acc: 0.1205
Epoch 6/200
40000/40000 [==============================] - 41s 1ms/sample - loss: 2.2283 - acc: 0.2221 - val_loss: 7.1401 - val_acc: 0.1146
Epoch 7/200
40000/40000 [==============================] - 41s 1ms/sample - loss: 2.2242 - acc: 0.2264 - val_l

KeyboardInterrupt: ignored

In [0]:
model.save('/content/TCR_Model_Normal.hdf5')